In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:
def save_in_file(items, file_name):
    file = open(file_name, "w")
    for item in items:
        file.write(item + "<myCustomTag>")
    file.close()

def get_from_files(file_name):
    file = open(file_name, "r")
    items = file.read().split("<myCustomTag>")
    file.close()

    return items[:-1]

In [3]:
def get_parsed_page(url):
    page = requests.get(url)
    return BeautifulSoup(page.text, 'html.parser')

def get_movie_ids(parsed_page, film_ids):
    films_links = parsed_page.find_all('a', class_='meta-title-link')
    for film_link in films_links:
        film_ids.append(film_link['href'].split('cfilm=')[1].split('.')[0])
    
    return film_ids

def get_top_20_movies_ids():
    base_URL = "https://www.allocine.fr/film/meilleurs/"
    parsed_page = get_parsed_page(base_URL)

    films_ids = []
    films_ids = get_movie_ids(parsed_page, films_ids)

    second_parsed_page = get_parsed_page(f"{base_URL}?page={2}")

    films_ids = get_movie_ids(second_parsed_page, films_ids)
    # films_links = parsed_page.find_all('a', class_='meta-title-link')
    # for film_link in films_links:
    #     films_ids.append(film_link['href'].split('cfilm=')[1].split('.')[0])
    
    return films_ids

def get_all_comments_page_links(parsed_page, URL):
    pagination = parsed_page.find('nav', class_='pagination')
    if pagination:
        last_page_element = pagination.find_all(['a', 'span'], class_='item')[-1]
        # last_page_link = last_page_element['href'] if last_page_element.name == 'a' else URL
        last_page_number = int(last_page_element.get_text())
        pages_urls = [f"{URL}?page={page}" for page in range(1, last_page_number + 1)]
    else:
        pages_urls = [URL]

    return pages_urls

def get_comments(parsed_page):
    comments = []
    for review in parsed_page.find_all('div', class_='content-txt review-card-content'):
        for spoiler in review.find_all('span', class_='spoiler-header'):
            spoiler.extract()

        comments.append(review.get_text(strip=True))
    
    return comments

def get_ratings(parsed_page):
        ratings = []
        # Récupérer les notes
        for review in parsed_page.find_all("div", class_="review-card-review-holder"):
            ratings.append(review.find("span", class_="stareval-note").get_text())
        
        return ratings

def scrap_comments(film_id):
    URL = f"https://www.allocine.fr/film/fichefilm-{film_id}/critiques/spectateurs/"
    parsed_movie_page = get_parsed_page(URL)

    pages_urls = get_all_comments_page_links(parsed_movie_page, URL)

    movie_comments = []
    movie_ratings = []
    page_number = 1
    for page_url in pages_urls:
        page_number+=1
        
        time.sleep(1)  # Attendre 1 seconde avant de faire la prochaine requête
        parsed_page = get_parsed_page(page_url)

        comments = get_comments(parsed_page)
        for comment in comments:
            movie_comments.append(comment)
        
        ratings = get_ratings(parsed_page)
        for rating in ratings:
            movie_ratings.append(rating)

        if(len(ratings) != len(comments)):
            print("=========================")

    return movie_comments, movie_ratings





In [4]:
film_ids = get_top_20_movies_ids()
print(len(film_ids))
print(film_ids)
all_comments = []
all_ratings = []
i = 0
for film_id in film_ids:
    print(i)
    i+=1
    movie_comments, movie_ratings = scrap_comments(film_id)

    for movie_comment in movie_comments:
        all_comments.append(movie_comment)
    for movie_rating in movie_ratings:
        all_ratings.append(movie_rating)
    
    # break # TODO: Scrap all
save_in_file(all_comments, "full_scrapped_movie_comments.txt")
save_in_file(all_ratings, "full_scrapped_movie_ratings.txt")


20
['10568', '9393', '22779', '4063', '1628', '11736', '39187', '10862', '2072', '115362', '2021', '10126', '19395', '1941', '114782', '24944', '190918', '27070', '39186', '135063']
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [5]:
binaryRating = []
for rating in get_from_files("full_scrapped_movie_ratings.txt"):
    if rating.split(",")[0] in ["3","4","5"]:
        binaryRating.append(1)
    else :
        binaryRating.append(0)

len(binaryRating)


43383